In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, ARDRegression, SGDRegressor, PassiveAggressiveRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from models import MLP, FONN1, FONN2, TREENN1, TREENN2

In [2]:
# Load the soybean dataset
raw_df = pd.read_excel("data/Soybean_V121.xlsx")
X = raw_df.values[0::1, 1:]
y = raw_df.values[0::1, 0]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X.shape, y.shape

((34213, 21), (34213,))

In [3]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27370, 21), (6843, 21), (27370,), (6843,))

In [4]:
# Function to train and evaluate a model
def train_evaluate_model(model, X_train, X_test, y_train, y_test):
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    train_time = end_time - start_time

    start_time = time.time()
    predictions = model.predict(X_test)
    end_time = time.time()
    comp_time = end_time - start_time

    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)

    return r2, mae, mse, train_time, comp_time


# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet(),
    "Bayesian Ridge Regression": BayesianRidge(),
    "ARD Regression": ARDRegression(),
    "SGD Regressor": SGDRegressor(),
    "Passive Aggressive Regressor": PassiveAggressiveRegressor(),
    "Support Vector Regression": SVR(),
    "MLP Regressor": MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost Regressor": XGBRegressor(random_state=42),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=42),
    "Bagging Regressor": BaggingRegressor(random_state=42),
    "ExtraTrees Regressor": ExtraTreesRegressor(random_state=42),
    "HistGradientBoosting Regressor": HistGradientBoostingRegressor(random_state=42),
    "Stacking Regressor": StackingRegressor(estimators=[
        ('lr', LinearRegression()),
        ('rf', RandomForestRegressor(n_estimators=10, random_state=42))
    ], final_estimator=Ridge()),
    "Voting Regressor": VotingRegressor(estimators=[
        ('lr', LinearRegression()),
        ('rf', RandomForestRegressor(n_estimators=10, random_state=42)),
        ('gb', GradientBoostingRegressor(random_state=42))
    ])
}

# Train and evaluate models
results = {}
for name, model in models.items():
    r2, mae, mse, fit_time, comp_time = train_evaluate_model(
        model, X_train, X_test, y_train, y_test)
    results[name] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                     "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Initialize MLP
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
epochs = 1000

mlp = MLP(input_dim, hidden_dim, output_dim,
          learning_rate=learning_rate, epochs=epochs)
r2, mae, mse, fit_time, comp_time = train_evaluate_model(
    mlp, X_train, X_test, y_train, y_test)
results["Custom MLP"] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                         "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

In [ ]:
# Initialize FONN1
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1
num_trees_input = 10
learning_rate = 0.001 # Reduced learning rate for FONN1
epochs = 4000 # Increase epochs for FONN1

fonn1 = FONN1(input_dim, hidden_dim, output_dim, num_trees_input,
              learning_rate=learning_rate, epochs=epochs)
r2, mae, mse, fit_time, comp_time = train_evaluate_model(
    fonn1, X_train, X_test, y_train, y_test)
results["FONN1"] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                    "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

In [ ]:
# Initialize FONN2
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1
num_trees_hidden = 10
epochs = 1000
learning_rate = 0.01

fonn2 = FONN2(input_dim, hidden_dim, output_dim, num_trees_hidden,
              learning_rate=learning_rate, epochs=epochs)
r2, mae, mse, fit_time, comp_time = train_evaluate_model(
    fonn2, X_train, X_test, y_train, y_test)
results["FONN2"] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                    "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

In [ ]:
# Initialize TREENN1
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
epochs = 1000

treenn1 = TREENN1(input_dim, hidden_dim, output_dim, learning_rate=learning_rate, epochs=epochs)
r2, mae, mse, fit_time, comp_time = train_evaluate_model(
    treenn1, X_train, X_test, y_train, y_test)
results["TREENN1"] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                      "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

In [ ]:
# Initialize TREENN2
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 1
learning_rate = 0.001
epochs = 4000 # Increase epochs

treenn2 = TREENN1(input_dim, hidden_dim, output_dim,
                  learning_rate=learning_rate, epochs=epochs)
r2, mae, mse, fit_time, comp_time = train_evaluate_model(
    treenn2, X_train, X_test, y_train, y_test)
results["TREENN2"] = {"R² Score": r2, "MAE": mae, "MSE": mse,
                      "Train Time (s)": fit_time, "Comp Time (s)": comp_time}

In [ ]:
# # Measure computational time and predict house prices using the decision trees in the hidden layer
# start_time = time.time()
# fonn2_tree_predictions = fonn2.tree_predict(X_test)
# end_time = time.time()
# fonn2_tree_comp_time = end_time - start_time

# fonn2_tree_r2 = r2_score(y_test, fonn2_tree_predictions)
# fonn2_tree_mae = mean_absolute_error(y_test, fonn2_tree_predictions)
# fonn2_tree_mse = mean_squared_error(y_test, fonn2_tree_predictions)

# results["Tree-based Predictions (FONN2)"] = {"R² Score": fonn2_tree_r2, "MAE": fonn2_tree_mae,
#                                              "MSE": fonn2_tree_mse, "Train Time (s)": fonn2_train_time, "Comp Time (s)": fonn2_tree_comp_time}

In [ ]:
# # Combine 10 decision trees and evaluate the ensemble model
# start_time = time.time()
# trees = [DecisionTreeRegressor(max_depth=5, random_state=i).fit(
#     X_train, y_train) for i in range(10)]
# end_time = time.time()
# ensemble_train_time = end_time - start_time

# start_time = time.time()
# ensemble_predictions = np.mean(
#     [tree.predict(X_test) for tree in trees], axis=0)
# end_time = time.time()
# ensemble_comp_time = end_time - start_time

# ensemble_r2 = r2_score(y_test, ensemble_predictions)
# ensemble_mae = mean_absolute_error(y_test, ensemble_predictions)
# ensemble_mse = mean_squared_error(y_test, ensemble_predictions)

# results["Ensemble of 10 Trees"] = {"R² Score": ensemble_r2, "MAE": ensemble_mae,
#                                    "MSE": ensemble_mse, "Train Time (s)": ensemble_train_time, "Comp Time (s)": ensemble_comp_time}

In [ ]:
# # Measure computational time and predict house prices using the decision tree in the hidden layer
# start_time = time.time()
# treenn2_tree_predictions = treenn2.tree_hidden.predict(X_test)
# end_time = time.time()
# treenn2_tree_comp_time = end_time - start_time

# treenn2_tree_r2 = r2_score(y_test, treenn2_tree_predictions)
# treenn2_tree_mae = mean_absolute_error(y_test, treenn2_tree_predictions)
# treenn2_tree_mse = mean_squared_error(y_test, treenn2_tree_predictions)

# results["Tree-based Predictions (TREENN2)"] = {"R² Score": treenn2_tree_r2, "MAE": treenn2_tree_mae,
#                                                "MSE": treenn2_tree_mse, "Train Time (s)": treenn2_train_time, "Comp Time (s)": treenn2_tree_comp_time}

In [ ]:
# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)

In [ ]:
# Get and print tree importances
tree_importances = fonn2.get_tree_importances()